In [73]:
import pandas as pd
import numpy as np

In [74]:
with open("product_list2.csv", "r") as fp:
    data_tgt = fp.readlines()

df = pd.read_csv("scraped list.csv")
    
list_tgt = [x.replace('"',"") for x in list(data_tgt)]
list_src = df['product-name'].values.tolist()#[:500]

In [75]:
def clean(s):
    d = {}
    s = s.lower()
    sl = s.split(" ")
    d.update({'brand': sl[0]})#, 'type': sl[1]})
    second = sl[2:]
    for part in second:
        if "mb/s" in part:
            d.update({'speed':part})
        if "gb" in part:
            d.update({'capacity':part[0:-1].replace('\n', "")})
            
    return d

In [76]:
prodict = {}
for l in list_tgt:
    prodict.update({l: clean(l)})

In [77]:
df = pd.DataFrame(prodict).T
df.head()

,brand,capacity,speed
KINGSTON CF CARD 4GB\n,kingston,4gb,NaN
KINGSTON CF CARD 8GB\n,kingston,8gb,NaN
KINGSTON SDC CLASS 10 80MB/S 128GB\n,kingston,128gb,80mb/s
KINGSTON SDC CLASS 10 80MB/S 16GB\n,kingston,16gb,80mb/s
KINGSTON SDC CLASS 10 80MB/S 256MB\n,kingston,NaN,80mb/s


In [78]:
df.isnull().sum()

brand        0
capacity     1
speed       19
dtype: int64

In [79]:
df['num'] = df.notnull().sum(axis=1)

In [80]:
df.head()

,brand,capacity,speed,num
KINGSTON CF CARD 4GB\n,kingston,4gb,NaN,2
KINGSTON CF CARD 8GB\n,kingston,8gb,NaN,2
KINGSTON SDC CLASS 10 80MB/S 128GB\n,kingston,128gb,80mb/s,3
KINGSTON SDC CLASS 10 80MB/S 16GB\n,kingston,16gb,80mb/s,3
KINGSTON SDC CLASS 10 80MB/S 256MB\n,kingston,NaN,80mb/s,2


In [81]:
type(df.iloc[0,0])

str

In [82]:
l = list_src[5]
print(l)
df_temp = df.drop(['num'], axis=1).applymap(lambda x: 1 if type(x) == str and x in l.lower() else 0)
key = df_temp.sum(axis=1).sort_values(ascending=False).index.tolist()[0]
num = df_temp.sum(axis=1).sort_values(ascending=False).values.tolist()[0]
print(key,num)
df['num'].loc[key]

Samsung Evo Plus Micro Sdxc Class 10 Uhs 1 64G Memory Card
SAMSUNG SDC EVO CLASS 10 100MB/S 128GB
 1


3

In [83]:
matchdict = {}
for s in list_src:
    df_temp = df.drop(['num'], axis=1).applymap(lambda x: 1 if type(x) == str and x in s.lower() else 0)
    key = df_temp.sum(axis=1).sort_values(ascending=False).index.tolist()[0]
    num = df_temp.sum(axis=1).sort_values(ascending=False).values.tolist()[0]
    if df['num'].loc[key] == num:
        matchdict.setdefault(key, []).append(s)

In [84]:
i = 0
tot = 0
for key, val in matchdict.items():
    print(i)
    print(key)
    tot += len(val)
    for v in val:
        print(v)
    print('-'*10)
    i += 1
print(i, tot)

0
SANDISK SDHC CARDS CLASS 10 - ULTRA - 80MB/S 32GB

Sandisk Ultra 32Gb Sdhc Uhs I U1 (Up To 80Mb/S Read) Memory Card
Sandisk 80Mb/S Class 10 Ultra Memory Micro Sd Card (Grey)32Gb  Intl
Sandisk Ultra 32Gb Microsdhc Ush 1 80Mb/S With Adapter
Sandisk Ultra 32Gb Sdhc Class10   80Mb/S
Sandisk Ultra 32Gb Microsdhc Uhs I (Up To 80Mb/S Read) Memory Cardw/Adapter
----------
1
SANDISK SDC CLASS 10 80MB/S SERIES 16GB

Sandisk Ultra 16Gb Class 10 Sd 80Mb/S
Sandisk Ultra 16Gb Microsdhc Uhs I Class 10 Card 80Mb/S With Sdadapter
Sandisk 16Gb 16G Ultra Sd Sdhc Full Hd Video Memory Keretad 80Mb/S*Class 10
Minicar Sandisk Ultra Microsdhc Uhs   I 16Gb High Speed 80Mb/Sclass 10 Sd Memory Card Adapter Blacksize:16Gb(Color:Black)(Int:16Gb)(Intl)   Intl
Sandisk Micro Sdhc Class 10 Card With Adapter 80Mb/S 16Gbsdsqunc 016G Gn6Ma (Export)
Sandisk Ultra High Speed Memory Card 80Mb/S Micro Sd Class 10 16Gb
Sandisk Ultra 16Gb 80Mb/S C10 Sdhc Uhs I Memory Card
----------
2
SANDISK CF EXTREME PRO 160MB/S SERIES 12